## Experimento 11 — Comparando Modelos (NB, SVM, KNN)
**Objetivo:** comparar modelos simples em um dataset clássico e acrescentar otimização dos hiperparâmetros

### Enunciado
1. Carregue o dataset **Iris** e separe em treino e teste (80/20) com `stratify`.
2. Realize uma **busca de modelo** simples comparando **LogisticRegression**, **SVC (linear)** e **RandomForest** com validação cruzada (k=5).  
3. **Tarefa Extra:** acrescente uma **busca de hiperparâmetros** com `GridSearchCV` para, pelo menos, dois modelos (ex.: SVC e RandomForest) usando `Pipeline` com `StandardScaler` quando necessário.  
4. Avalie no conjunto de teste o melhor modelo encontrado e **salve o modelo** com `joblib.dump` para simular deploy.


In [11]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn (para os experimentos de ML)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# Opção para gráficos inline (no Jupyter/Colab)
# %matplotlib inline  # Descomente no Jupyter clássico se necessário

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [12]:
# Dados
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Mesmo esquema de CV da fase com GridSearch
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Mesmos candidatos do código-base, em Pipeline quando necessário
candidates = {
    "LogReg": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=2000, random_state=RANDOM_STATE))
    ]),
    "SVC_linear": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="linear", random_state=RANDOM_STATE))
    ]),
    "RF": RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
}

# Avaliação com cross_validate para obter média e desvio
rows = []
for name, model in candidates.items():
    cv_res = cross_validate(
        model,
        X_train, y_train,
        cv=cv,
        scoring="accuracy",
        return_train_score=False,
        n_jobs=-1
    )
    cv_mean = cv_res["test_score"].mean()
    cv_std  = cv_res["test_score"].std()

    # Treina no treino inteiro e mede no teste (coerente com a etapa de GS)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)

    rows.append({
        "model": name,
        "cv_mean_acc": cv_mean,
        "cv_std_acc": cv_std,
        "test_acc": test_acc
    })

df_base = pd.DataFrame(rows).sort_values(by="cv_mean_acc", ascending=False).reset_index(drop=True)
print(df_base)

# Escolhe o melhor por CV médio (mesmo critério da etapa com GridSearch)
best_name = df_base.iloc[0]["model"]
best_model = candidates[best_name]
print(f"\n== Melhor por CV (sem GridSearch): {best_name} ==")

# Reavalia melhor modelo para relatar detalhes (já treinado acima, mas reforçamos)
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)
print(f"Acurácia no teste: {accuracy_score(y_test, y_pred_best):.3f}\n")
print("Relatório de classificação:\n", classification_report(y_test, y_pred_best))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_best))


        model  cv_mean_acc  cv_std_acc  test_acc
0  SVC_linear     0.975000    0.033333  1.000000
1      LogReg     0.958333    0.026352  0.933333
2          RF     0.950000    0.031180  0.900000

== Melhor por CV (sem GridSearch): SVC_linear ==
Acurácia no teste: 1.000

Relatório de classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Matriz de confusão:
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]


### Adaptação (Extra)
- Use `GridSearchCV` com `cv=StratifiedKFold(5)`.
- Defina `param_grid` apropriados. Exemplos:
  - **SVC**: `C=[0.1,1,10]`, `kernel=['linear','rbf']`, `gamma=['scale','auto']`
  - **RandomForest**: `n_estimators=[100,300]`, `max_depth=[None,5,10]`, `min_samples_split=[2,5]`
- Envolva SVC em `Pipeline(StandardScaler(), SVC(...))` para evitar *data leakage*.
- Relate `best_params_`, `best_score_` e avalie no teste.
- Por fim, salve o melhor modelo com `joblib.dump`.


In [9]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# ===== Mesmos classificadores do código-base, agora com Grids =====
# LogReg + scaler
pipe_logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, random_state=RANDOM_STATE))
])
grid_logreg = {
    "clf__C": [0.1, 1, 3, 10],
    "clf__penalty": ["l2"],                # (l1 requer solver 'liblinear' e sem multi_class='multinomial')
    "clf__solver": ["lbfgs", "liblinear"], # ambos funcionam com l2
}

# SVC linear + scaler (mantendo kernel linear como no seu base)
pipe_svc_lin = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel="linear", random_state=RANDOM_STATE))
])
grid_svc_lin = {
    "clf__C": [0.1, 1, 3, 10]
}

# RandomForest (não precisa de scaler)
rf = RandomForestClassifier(random_state=RANDOM_STATE)
grid_rf = {
    "n_estimators": [100, 200, 400],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5]
}

# Empacotar tudo
search_space = [
    ("LogReg",    pipe_logreg, grid_logreg),
    ("SVC_linear", pipe_svc_lin, grid_svc_lin),
    ("RF",        rf,          grid_rf),
]

results = []
best_global = None
best_global_name = None
best_global_cv = -np.inf

for name, estimator, param_grid in search_space:
    gs = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring="accuracy",
        n_jobs=-1,
        refit=True,
        return_train_score=False,
    )
    gs.fit(X_train, y_train)

    # Avaliação em teste
    y_pred = gs.best_estimator_.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)

    results.append({
        "model": name,
        "best_params": gs.best_params_,
        "cv_mean_acc": gs.best_score_,
        "test_acc": test_acc
    })

    # Guardar "melhor por CV"
    if gs.best_score_ > best_global_cv:
        best_global_cv = gs.best_score_
        best_global = gs.best_estimator_
        best_global_name = name

# Mostrar ranking
df_results = pd.DataFrame(results).sort_values(by="cv_mean_acc", ascending=False).reset_index(drop=True)
print(df_results)

print(f"\n== Melhor por CV: {best_global_name} ==")
y_pred_best = best_global.predict(X_test)
print(f"Acurácia no teste: {accuracy_score(y_test, y_pred_best):.3f}\n")
print("Relatório de classificação:\n", classification_report(y_test, y_pred_best))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_best))

# Salvar melhor modelo (simulação de deploy)
joblib.dump(best_global, "melhor_modelo_gridsearch.joblib")
print("\nModelo salvo em: melhor_modelo_gridsearch.joblib")

        model                                        best_params  cv_mean_acc  \
0  SVC_linear                                    {'clf__C': 0.1}     0.975000   
1      LogReg  {'clf__C': 3, 'clf__penalty': 'l2', 'clf__solv...     0.966667   
2          RF  {'max_depth': None, 'min_samples_split': 5, 'n...     0.966667   

   test_acc  
0  0.933333  
1  0.966667  
2  0.966667  

== Melhor por CV: SVC_linear ==
Acurácia no teste: 0.933

Relatório de classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

Matriz de confusão:
 [[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]

Modelo salvo em: melhor_modelo_gridsearch.joblib
